In [22]:
import pandas as pd
from glob import glob

In [23]:
#Ok, this is a mess, but we need to do it this way.
#We're going to create a fixed list of all of the different industries we want to use,
#excluding any top-level codes that have a sub-code.
#That'd be a pain to code, so we're just going to drop them here,
#with a map for renaming the fields later so they're more manageable.


INDUSTRY_NAMES  = {
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Agriculture, forestry, fishing and hunting, and mining: - Agriculture, forestry, fishing and hunting" : "Agriculture, forestry, fishing and hunting",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Agriculture, forestry, fishing and hunting, and mining: - Mining, quarrying, and oil and gas extraction" : "Mining, quarrying, and oil and gas extraction",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Construction:" : "Construction",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Manufacturing:" : "Manufacturing",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Wholesale trade:" : "Wholesale trade",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Retail trade:" : "Retail trade",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Transportation and warehousing, and utilities: - Transportation and warehousing" : "Transportation and warehousing",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Transportation and warehousing, and utilities: - Utilities" : "Utilities",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Information:" : "Information",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Finance and insurance, and real estate and rental and leasing: - Finance and insurance" : "Finance and insurance",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Finance and insurance, and real estate and rental and leasing: - Real estate and rental and leasing" : "Real estate and rental and leasing",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Professional, scientific, and management, and administrative and waste management services: - Professional, scientific, and technical services" : "Professional, scientific, and technical services",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Professional, scientific, and management, and administrative and waste management services: - Management of companies and enterprises" : "Management of companies and enterprises",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Professional, scientific, and management, and administrative and waste management services: - Administrative and support and waste management services" : "Administrative and support and waste management services",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Educational services, and health care and social assistance: - Educational services" : "Educational services",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Educational services, and health care and social assistance: - Health care and social assistance" : "Health care and social assistance",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Arts, entertainment, and recreation, and accommodation and food services: - Arts, entertainment, and recreation" : "Arts, entertainment, and recreation",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Arts, entertainment, and recreation, and accommodation and food services: - Accommodation and food services" : "Accommodation and food services",
    "Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings - Public administration:" : "Public administration"
}

In [24]:
#Pull in data
datasets = {}

short_year = ['07','17']
for year in [2007, 2017]:
    print(year)
    data = pd.DataFrame()
    
    #Pull in wage data
#     wage_str = {
#         2007: 'Estimate; Total:',
#         2017: 'Estimate; Total:'
#     }
    wages = pd.read_csv(glob('data/wages/' + str(year) + '_B24041/*.csv')[0],header=1)
    data = pd.concat([
        data,
        wages[['Estimate; Total:', 'Id2', 'Geography']]                 
    ], axis=1)
    data.rename(columns={
        'Estimate; Total:' : 'Median earnings ' + str(year)
    }, inplace=True)
        
        
    #Pull in population data for filtering out
    #cities w/ <100,000 people
    population = pd.read_csv(glob('data/population/' + str(year) + '/*.csv')[0], header=1)
    print("Population rows: ", len(population))
    
    #Merge
    data = data.merge(population[['Estimate; Total', 'Id2']], on='Id2', how='inner')
    data.rename(columns={
        'Estimate; Total' : 'Population ' + str(year)
    },inplace=True)
    
    #Push out dataframe
    datasets[year] = data
        
#Pull in employment data by industry
employment_2017 = pd.read_csv(glob('data/employment/2017/*.csv')[0],header=1)
employment_2007 = pd.read_csv(glob('data/employment/2007/*.csv')[0],header=1)

#Rename columns so we can identify them later
employment_2017.columns = [str(col) + ' 2017' for col in employment_2017.columns]
employment_2007.columns = [str(col) + ' 2007' for col in employment_2007.columns]
employment_2017.rename(columns={'Id 2017' : 'Id', 'Id2 2017' : 'Id2', 'Geography 2017': 'Geography'}, inplace=True)
employment_2007.rename(columns={'Id 2007' : 'Id', 'Id2 2007' : 'Id2', 'Geography 2007': 'Geography'}, inplace=True)

employment = employment_2007.merge(employment_2017.drop(['Id', 'Geography'], axis=1), on='Id2', how='inner') 

print("Employment 2017 rows: ", len(employment_2007))
print("Employment 2007 rows: ", len(employment_2017))

2007
Population rows:  513
2017
Population rows:  608
Employment 2017 rows:  513
Employment 2007 rows:  608


OK, so it looks like 95 additional cities qualified in the ten years in between. What do?

In [25]:
# datasets[2007].drop('Geography', axis=1, inplace=True)
SUBSTRINGS = [
    'Male;',
    'Female;',
    'Margin of Error;'
]
data = datasets[2007].merge(datasets[2017], on='Id2', how='inner')
data = data.merge(employment, on='Id2', how='inner')
data.dropna(axis=0, how='any', inplace=True)
data = data.loc[
    ~(data['Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings 2007'] == 'N')
]
data = data.loc[
    data['Population 2017'] > 100000
]
data
for col in data.columns:
    if any(substring.lower() in col.lower() for substring in SUBSTRINGS):
        data.drop(col,axis=1,inplace=True)

In [26]:
data

,Median earnings 2007,Id2,Geography_x,Population 2007,Median earnings 2017,Geography_y,Population 2017,Id,Geography,"Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings 2007",...,"Total; Estimate; Professional, scientific, and management, and administrative and waste management services: - Management of companies and enterprises 2017","Total; Estimate; Professional, scientific, and management, and administrative and waste management services: - Administrative and support and waste management services 2017","Total; Estimate; Educational services, and health care and social assistance: 2017","Total; Estimate; Educational services, and health care and social assistance: - Educational services 2017","Total; Estimate; Educational services, and health care and social assistance: - Health care and social assistance 2017","Total; Estimate; Arts, entertainment, and recreation, and accommodation and food services: 2017","Total; Estimate; Arts, entertainment, and recreation, and accommodation and food services: - Arts, entertainment, and recreation 2017","Total; Estimate; Arts, entertainment, and recreation, and accommodation and food services: - Accommodation and food services 2017","Total; Estimate; Other services, except public administration 2017",Total; Estimate; Public administration 2017
0,30975,107000,"Birmingham city, Alabama",206215,32363,"Birmingham city, Alabama",212744,1600000US0107000,"Birmingham city, Alabama",60496,...,0,2900,16130,4829,11301,6582,1703,4879,2961,3479
3,40028,137000,"Huntsville city, Alabama",167121,45940,"Huntsville city, Alabama",195369,1600000US0137000,"Huntsville city, Alabama",47733,...,123,2139,10750,3965,6785,4369,624,3745,2288,6868
4,31623,150000,"Mobile city, Alabama",189476,39526,"Mobile city, Alabama",190273,1600000US0150000,"Mobile city, Alabama",52358,...,44,2809,15602,5605,9997,5833,975,4858,2201,2990
5,36379,151000,"Montgomery city, Alabama",200834,37876,"Montgomery city, Alabama",199525,1600000US0151000,"Montgomery city, Alabama",59922,...,44,1676,10524,4365,6159,4757,938,3819,2834,7106
6,35586,177256,"Tuscaloosa city, Alabama",77430,41227,"Tuscaloosa city, Alabama",100296,1600000US0177256,"Tuscaloosa city, Alabama",17302,...,0,1449,9762,4344,5418,3260,155,3105,1066,402
7,46518,203000,"Anchorage municipality, Alaska",279671,53139,"Anchorage municipality, Alaska",294356,1600000US0203000,"Anchorage municipality, Alaska",85223,...,513,2641,24784,6222,18562,9617,736,8881,6085,13818
9,49753,412000,"Chandler city, Arizona",227940,55938,"Chandler city, Arizona",253448,1600000US0412000,"Chandler city, Arizona",86666,...,208,5016,19185,7264,11921,6184,1654,4530,3133,3648
10,51350,427400,"Gilbert town, Arizona",195090,58563,"Gilbert town, Arizona",242362,1600000US0427400,"Gilbert town, Arizona",64787,...,530,3950,19280,7595,11685,5199,1709,3490,3174,4074
11,37213,427820,"Glendale city, Arizona",235940,39578,"Glendale city, Arizona",246708,1600000US0427820,"Glendale city, Arizona",73704,...,0,4911,19160,6036,13124,5349,1623,3726,3053,4141
12,35606,446000,"Mesa city, Arizona",479415,41472,"Mesa city, Arizona",496395,1600000US0446000,"Mesa city, Arizona",150898,...,151,11045,32426,11619,20807,10124,2619,7505,8120,5482


In [28]:
data.to_csv('output/merged_data.csv')

In [ ]:
employment

In [ ]:
counter = 0
PRESENT_2017_NOT_2007 = []
for value in datasets[2017]['Id2'].unique():
    try:
        assert(value in datasets[2007]['Id2'].unique())
    except AssertionError:
        PRESENT_2017_NOT_2007.append(value)
        print(value)
        
print(counter)

In [ ]:
len(set(datasets[2017]['Id2']).intersection(datasets[2007]['Id2']))

We want to see how many cities that had >100,000 people in 2017 also had >100,000 in 2007.

In [ ]:
len(data.loc[ 
    (data['Population 2017'] > 100000) 
])

In [ ]:
len(datasets[2017].loc[datasets[2017]['Population 2017'] > 100000])

In [ ]:
len(datasets[2017])

In [ ]:
len(data.loc[
    data['Population 2017'] > 100000
])

In [ ]:
list(data)

In [ ]:
datasets[2007]

In [29]:
#hack in Kyle's earnings estimates
earnings = pd.read_excel('data/earnings.xlsx')

In [30]:
earnings

,Id,Id2,Geography,2017,2007,Change,Population
0,1600000US5305210,5305210,"Bellevue city, Washington",100020,66006,0.515317,144459
1,1600000US4134100,4134100,"Hillsboro city, Oregon",59466,41178,0.444121,106903
2,1600000US0668252,668252,"San Mateo city, California",87371,60759,0.437993,104733
3,1600000US0667000,667000,"San Francisco city, California",80812,56256,0.436505,884363
4,1600000US5363000,5363000,"Seattle city, Washington",72052,51868,0.389142,724764
5,1600000US2511000,2511000,"Cambridge city, Massachusetts",72341,52185,0.386241,113631
6,1600000US0611194,611194,"Carlsbad city, California",90009,65201,0.380485,115326
7,1600000US4848072,4848072,"Midland city, Texas",56414,41289,0.366320,136090
8,1600000US1150000,1150000,"Washington city, District of Columbia",70725,51856,0.363873,693972
9,1600000US4827684,4827684,"Frisco city, Texas",83053,61354,0.353669,177281


In [32]:
data = data.merge(earnings[['Id', 2007, 2017, 'Change']], on='Id', how='inner')

In [33]:
data

,Median earnings 2007,Id2,Geography_x,Population 2007,Median earnings 2017,Geography_y,Population 2017,Id,Geography,"Total; Estimate; Full-time, year-round civilian employed population 16 years and over with earnings 2007",...,"Total; Estimate; Educational services, and health care and social assistance: - Educational services 2017","Total; Estimate; Educational services, and health care and social assistance: - Health care and social assistance 2017","Total; Estimate; Arts, entertainment, and recreation, and accommodation and food services: 2017","Total; Estimate; Arts, entertainment, and recreation, and accommodation and food services: - Arts, entertainment, and recreation 2017","Total; Estimate; Arts, entertainment, and recreation, and accommodation and food services: - Accommodation and food services 2017","Total; Estimate; Other services, except public administration 2017",Total; Estimate; Public administration 2017,2007,2017,Change
0,30975,107000,"Birmingham city, Alabama",206215,32363,"Birmingham city, Alabama",212744,1600000US0107000,"Birmingham city, Alabama",60496,...,4829,11301,6582,1703,4879,2961,3479,30975,32363,0.044810
1,40028,137000,"Huntsville city, Alabama",167121,45940,"Huntsville city, Alabama",195369,1600000US0137000,"Huntsville city, Alabama",47733,...,3965,6785,4369,624,3745,2288,6868,40028,45940,0.147697
2,31623,150000,"Mobile city, Alabama",189476,39526,"Mobile city, Alabama",190273,1600000US0150000,"Mobile city, Alabama",52358,...,5605,9997,5833,975,4858,2201,2990,31623,39526,0.249913
3,36379,151000,"Montgomery city, Alabama",200834,37876,"Montgomery city, Alabama",199525,1600000US0151000,"Montgomery city, Alabama",59922,...,4365,6159,4757,938,3819,2834,7106,36379,37876,0.041150
4,35586,177256,"Tuscaloosa city, Alabama",77430,41227,"Tuscaloosa city, Alabama",100296,1600000US0177256,"Tuscaloosa city, Alabama",17302,...,4344,5418,3260,155,3105,1066,402,35586,41227,0.158517
5,46518,203000,"Anchorage municipality, Alaska",279671,53139,"Anchorage municipality, Alaska",294356,1600000US0203000,"Anchorage municipality, Alaska",85223,...,6222,18562,9617,736,8881,6085,13818,46518,53139,0.142332
6,49753,412000,"Chandler city, Arizona",227940,55938,"Chandler city, Arizona",253448,1600000US0412000,"Chandler city, Arizona",86666,...,7264,11921,6184,1654,4530,3133,3648,49753,55938,0.124314
7,51350,427400,"Gilbert town, Arizona",195090,58563,"Gilbert town, Arizona",242362,1600000US0427400,"Gilbert town, Arizona",64787,...,7595,11685,5199,1709,3490,3174,4074,51350,58563,0.140467
8,37213,427820,"Glendale city, Arizona",235940,39578,"Glendale city, Arizona",246708,1600000US0427820,"Glendale city, Arizona",73704,...,6036,13124,5349,1623,3726,3053,4141,37213,39578,0.063553
9,35606,446000,"Mesa city, Arizona",479415,41472,"Mesa city, Arizona",496395,1600000US0446000,"Mesa city, Arizona",150898,...,11619,20807,10124,2619,7505,8120,5482,35606,41472,0.164748


In [35]:
data.to_csv('merged_data_corrected.csv')